# Генерация данных #

### Загрузка пакетов ###

In [10]:
#обработка музыки
import librosa

# загрузка mp3 файлов
import audioread.ffdec  # Use ffmpeg decoder
import ffmpeg

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = (20,10)

#файлы в директории
from glob import glob

#воспроизведение музыки
from IPython.display import Audio

#сохранение музыки
import soundfile as sf
import json
import csv
import os

In [91]:
#класс искажения звука
class MusicEffectGen:
    def __init__(self, noise_types=["change_volume_sin"], t = 7.5, sr = 22050):
        """
        Инициализация объекта класса MusicEffectGen
        
        Параметры:
            noise_type (str) - тип звукового эффекта
        
        Возвращает:
            None
            
        Примечание:
            self.noise_func - функция классса MusicEffectGen, изменяющая звуковые ряды
        """
        self.t = t
        self.sr = sr
        self.sample_len = sample_len = librosa.time_to_samples(self.t, sr = self.sr)
        
        self.noise_funcs = {
           "change_volume_c": self.change_volume_c,
            "change_volume_sin": self.change_volume_sin,
            "change_volume_fading": self.change_volume_fading,
            "change_volume_increase": self.change_volume_increase,
            "add_some_noise": self.add_some_noise
        }
        
        
        self.noise_func = None
        self.samples = None
        self.noised_samples = None
        self.noises = None
        
        self.chose_noise_func(noise_types)
        self.load_noises()
        
        
    def chose_noise_func(self, noise_types):
        """
        Изменяет тип шума на noise_type
        
        Параметры:
            noise_type (str) - тип звукового эффекта
        
        Возвращает:
            None
            
        Примечание:
        """
        if noise_types == 'all':
            self.noise_func = list(self.noise_funcs.values())
        else:
            self.noise_func = [self.noise_funcs[noise_type] for noise_type in noise_types]
    
    
    def change_volume_c(self, index, volume_level=0.5):
        """
        Изменение громкости звукового ряда в соответствии с постоянным коэффициентом volume_level
        
        Параметры:
            index (int) - номер сэмпла для данного эффекта
            volume_level (number) - от 0 до inf (но не стоит). уровень громкости
        
        Возвращает:
            noised_samples (np.ndarray) - зашумленные нарезанные музыкальные фрагменты
            
        Примечание:
        """
        self.noised_samples[index] = self.noised_samples[index] * volume_level
        
        
    def change_volume_sin(self, index, volume_level=20):
        """
        Изменение громкости звукового ряда в соответствии со значениями функции синуса,
            взятыми с равным шагом из интервала (-volume_level / 2, volume_level / 2)
        
        Параметры:
            index (int) - номер сэмпла для данного эффекта
            volume_level (number) - от 0 до inf (но не стоит). уровень громкости
        
        Возвращает:
            noised_samples (np.ndarray) - зашумленные нарезанные музыкальные фрагменты
            
        Примечание:
            Параметр volume_level влияет на частоту колебаний громкости звука. При volume_level=20
                и длине муз. фрагментов 7.5 сек примерно 3 периода затихания-нарастания громкости
        """
        self.noised_samples[index] = self.noised_samples[index] *\
            np.array([(np.sin(x) + 1)*0.5 for x in \
                      np.arange(-volume_level / 2, volume_level / 2, volume_level / self.sample_len)])
    
    
    def change_volume_fading(self, index):
        """
        Затихание громкости звукового ряда от начальной громности до 0
        
        Параметры:
            index (int) - номер сэмпла для данного эффекта
        
        Возвращает:
            None
            
        Примечание:
        """
        self.noised_samples[index] = self.noised_samples[index] * np.arange(1, 0, -1 / self.sample_len)
    
    
    def change_volume_increase(self, index):
        """
        Нарастание громкости звукового ряда от 0 до начальной громности
        
        Параметры:
            index (int) - номер сэмпла для данного эффекта
        
        Возвращает:
            None
            
        Примечание:
        """
        self.noised_samples[index] = self.noised_samples[index] * np.arange(0, 1, 1 / self.sample_len)
    
    
    def add_some_noise(self, index, volume_level=0.5):
        """
        Накладывает на элемент noised samples шум из заранее загруженных с громкость volume level
        
        Параметры:
            index - индекс сэмпла для изменения
            volume_level (number) - от 0 до inf (но не стоит). уровень громкости
            
        Возвращает:
            None
            
        Примечание:
        """
        noised_sample = self.noised_samples[index]
        #для подгонки длины
        n = len(noised_sample)
        
        #выбор случайного шума из загруженных
        noise = np.random.choice(self.noises)

        noised_sample = noise[:n] * volume_level + noised_sample
        self.noised_samples[index] = noised_sample
    
    def load_noises(self, noises_dir="./noises_samples"):
        """
        Загружает возможные шумовые файлы в класс. Проходит по всем шумам, закливает их, 
        если они короче 7.5 секунд, сохраняет.
        
        Параметры:
            noises_dir - директория с шумами
        
        Возвращает:
            None
        
        Примечание:
        
        """
        noises_names = glob(noises_dir+ "\*.mp3")
        self.noises = []
        
        
        for noise_name in noises_names:
            _y, _sr = librosa.load(audioread.ffdec.FFmpegAudioFile(noise_name))
            #_y = np.repeat(_y, np.ceil(self.sample_len / len(_y)))
            _y = np.array([_y for i in range(int(np.ceil(self.sample_len / len(_y))))]).flatten()
            self.noises.append(_y)
            
    def load_samples(self, samples):
        """
        Загружает сэмплы в класс.
        
        Параметры:
            samples - сэмлы
        
        Возвращает:
            None
        
        Примечание:
        
        """
        self.samples = samples
        
    def generate_noised_samples(self):
        """
        Загружает сэмплы в класс.
        
        Параметры:
            None
        
        Возвращает:
            None
        
        Примечание:
        
        """
        self.noised_samples = self.samples
    
    def add_random_noises(self, samples):
        self.load_samples(samples)
        self.generate_noised_samples()
        
        for i in range(len(self.noised_samples)):
            func = np.random.choice(self.noise_func)
            func(i)
        return self.noised_samples

In [92]:
#класс генератора данных 
class DataGenerator:
    def __init__(self, audio_path=None):
        """
        Инициализация объекта класса DataGenerator
        
        Параметры:
            audio_path (str) - путь к аудиофайлу
            
        Возвращает:
            None
            
        Примечание:
            self.path (str) - путь к файлу
            self.name (str) - название музыкальной композиции
            self.y (np.ndarray) - звуковой временной ряд
            self.sr (number) - частота дискретизации y
            self.samples (np.ndarray) - нарезанные музыкальные фрагменты
            self.noised_samples (np.ndarray) - зашумлённые музыкальные фрагменты
            self.number_of_samples (number) - количество нарезанных фрагментов
            self.noise_func (MusicEffectGen) - объект класса MusicEffectGen, содержащий в себе различные функции шума
        """
        
        self.path = audio_path
        self.name = None
        self.y = None
        self.sr = None
        self.samples = None
        self.number_of_samples = None
        self.noised_samples = None
        self.noises_class = MusicEffectGen()
        
        
    def read_audio(self, audio_path):
        """
        Чтение аудиофайла, лежащего в audio_path
        
        Параметры:
            audio_path (str) - путь к аудиофайлу
            
        Возвращает:
            None
            
        Примечание:
            Инициализирует атрибуты класса:
                y (np.ndarray) - звуковой временной ряд
                sr (int > 0) - частота дискретизации y
                name (str) - название текущего файла
        """
        self.y, self.sr = librosa.load(audioread.ffdec.FFmpegAudioFile(audio_path))
        
        name_with_extension = audio_path.split('\\')[-1]
        self.name = name_with_extension.split('.')[0]
        
        
    def to_samples(self, t = 7.5):
        """
        Нарезает звуковой временной ряд self.y на дорожки длиной t секунд
        
        Параметры:
            t (number or np.ndarray) - длина нарезки аудиофрагментов
            
        Возвращает:
            None
            
        Примечание:
            Инициализирует атрибут класса:
                self.music_fragments (np.ndarray) - массив, содержащий звуковые временные
                    ряды длины t
            
            Нарезанные фрагменты не перекрывают друг друга, за исключением, может быть, последнего,
                который подгоняется под длину t
        """
        self.number_of_samples = librosa.time_to_samples(t, sr = self.sr)
        _music_fragments = []
        
        #нарезка
        for i in range(int(np.ceil(self.y.shape[0] / self.number_of_samples))):
            _music_fragments.append(self.y[i * self.number_of_samples: (i + 1) * self.number_of_samples])
            
        #приведение последнего объекта к 7.5 секундам
        _music_fragments[-1] = np.append(_music_fragments[-1], np.zeros((self.number_of_samples - _music_fragments[-1].shape)))
        
        self.samples = np.array(_music_fragments)
        
    
    def get_samples(self):
        """
        Выгружает нарезанные фрагменты длины t
        
        Параметры:
            None
            
        Возвращает:
            self.samples (np.ndarray) - массив, содержащий звуковые временные ряды длины t
            
        Примечание:
            Использует метод to_samples
        """
        return self.samples
   

    def add_noise(self, noise_functions=["change_volume_sin"]):
        
        """
        Добавляет шум к нарезанным музыкальным фрагментам samples
            
        Параметры:
            noise_function (str) - функция, которая накладывает шум, работает от списка сэмплов
            
        Возвращает:
            None
            
        Примечание:
        """
        #выбирает нужные функции шума и запускает их
        self.noises_class.chose_noise_func(noise_functions)
        self.noised_samples = self.noises_class.add_random_noises(self.samples)
        
        
    def save_samples(self, save_dir):
        """
        Сохраняет нарезанные музыкальные фрагменты samples
            в директории save_dir
            
        Параметры:
            save_dir (str) - директория, куда сохраняются файлы
            
        Возвращает:
            None
            
        Примечание:
            Сохраненные файлы называются по следующему принципу:
                имя-исходного-файла_номер-фрагмента.wav
        """
        for i in range(len(self.samples)):
            sf.write(save_dir + self.name + "_" + str(i) + ".wav", self.samples[i], self.sr)
           
        
    def save_noised_samples(self, save_dir):
        """
        Сохраняет нарезанные музыкальные фрагменты samples с добавленным шумом в степени noise_level
            в директории save_dir
            
        Параметры:
            save_dir (str) - директория, куда сохраняются файлы
            noise_level (number) - степень шума. от 0 до 1
            
        Возвращает:
            None
            
        Примечание:
            Сохраненные файлы называются по следующему принципу:
                noised_степень-шума_имя-исходного-файла_номер-фрагмента.wav
        """
        
        for i in range(len(self.noised_samples)):
            filename = "noised_" + "_" + self.name + "_" + str(i) + ".wav"
            directory_name = save_dir + self.name + "\\"

            if not os.path.exists(directory_name):
                os.makedirs(directory_name)
            sf.write( directory_name + filename , self.noised_samples[i], self.sr)
            
            
    def save_to_json(self, save_dir):
        """
        Сохраняет нарезанные музыкальные фрагменты samples
            в json файле save_dir
            
        Параметры:
            save_dir (str) - json файл, куда сохраняются данные
            
        Возвращает:
            None
            
        Примечание:
            Сохраненные файлы называются по следующему принципу:
                [{
                    init_dict:
                        {
                        name: str
                        sr: int > 0
                        noised: bool
                        noise_level: number
                        },
                    samples: list of lists of numbers
                 } , ...]
        """
        with open(save_dir + self.name + '.json', "w") as file:
            _data = {'init_dict': {'name': self.name, 'sr': self.sr}, 
                          'samples': [list(_) for _ in self.samples]}
            file.seek(0)
            json.dump(_data, file)
            
    def save_noised_to_json(self, save_dir):
        """
        Сохраняет нарезанные музыкальные фрагменты samples
            в json файле save_dir
            
        Параметры:
            save_dir (str) - json файл, куда сохраняются данные
            
        Возвращает:
            None
            
        Примечание:
            Сохраненные файлы называются по следующему принципу:
                [{
                    init_dict:
                        {
                        name: str
                        sr: int > 0
                        noised: bool
                        noise_level: number
                        },
                    samples: list of lists of numbers
                 } , ...]
        """
        with open(save_dir + self.name + '.json', "w") as file:
            _data = {'init_dict': {'name': self.name, 'sr': self.sr}, 
                          'samples': [list(_) for _ in self.noised_samples]}
            file.seek(0)
            json.dump(_data, file)
    
    def save_to_csv(self, save_dir):
        """
        Сохраняет нарезанные музыкальные фрагменты csv файле
            
        Параметры:
            save_dir (str) - json файл, куда сохраняются данные
            
        Возвращает:
            None
            
        Примечание:
        
        """
        with open(save_dir + self.name + '.csv', 'w', newline='') as file:
            spamwriter = csv.writer(file)
            spamwriter.writerow([self.name, self.sr])
            for sample in self.noised_samples:
                spamwriter.writerow(sample)
    
    def generate_noised_data(self, audio_path, save_dir, _type = 'sample', noise_functions = 'all'):
        """
        Генерирует зашумлённое аудио по пути audio_path,
        сохраняет в директорию по пути save_dir
        
            
        Параметры:
            audio_path (str) - путь до музыкального файла
            save_dir (str) - директория сохранения
            _type(str) - csv или sample, показывает тип сохранения данных
            
        Возвращает:
            None
            
        Примечание:
       
        """
        self.read_audio(audio_path)
        self.to_samples()
        self.add_noise(noise_functions=noise_functions)
        if _type == 'sample':
            self.save_noised_samples(save_dir)
        elif _type == 'csv':
            self.save_to_csv(save_dir)

In [93]:
def generate_data(dataGen, audio_path, save_dir, _type = 'sample'):
    musicfiles = glob(audio_path+ "\*.mp3")
    for song in musicfiles:
        dataGen.generate_noised_data(song, save_dir, _type, noise_functions = 'all')

In [94]:
ex_dg = DataGenerator()

In [95]:
audio_path = '.\\music'
save_dir = '.\\wav_data\\'
noise_dir = '.\\noises_samples'
generate_data(ex_dg, audio_path, save_dir)

<ipython-input-91-bcb0d056a2be>:141: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  noise = np.random.choice(self.noises)
